In [27]:
import pandas as pd
import requests
from pprint import pprint
import psycopg as pg
from datetime import datetime
import calendar
import numpy as np

In [50]:
cities = [
    {"q": "Bogota", "latitude": 4.7110, "longitude": -74.0721},
    {"q": "Antananarivo", "latitude": -18.8792, "longitude": 47.5079},
    {"q": "Taipei", "latitude": 25.0330, "longitude": 121.5654},
    {"q": "Kiev", "latitude": 50.4501, "longitude": 30.5234},
    {"q": "Paris", "latitude": 48.8566, "longitude": 2.3522},
    {"q": "Munich", "latitude": 48.1351, "longitude": 11.5820},
    {"q": "Tokyo", "latitude": 35.6895, "longitude": 139.6917},
    {"q": "Venise", "latitude": 45.4408, "longitude": 12.3155},
    {"q": "Rio de Janeiro", "latitude": -22.9068, "longitude": -43.1729},
]

open_meteo = "https://archive-api.open-meteo.com/v1/archive"

def extract_data_test(cities):
    for city in cities :
        params = {
             'q' : city['q'],
            'appid' : "c5e106cfed4654aa7d4d56a055c7dad8",
            'units' : 'metric'
        }
        open_meteo_params = {
        'latitude' : city['latitude'],
        'longitude' : city['longitude'],
        'start_date' : "2025-06-25",
        'end_date' : "2025-06-26",
        'hourly' : "temperature_2m"
        }
       # response = requests.get(f"https://api.openweathermap.org/data/2.5/weather",params=params)
        response_open_meteo = requests.get(open_meteo,open_meteo_params)
        #pour open weather reel time
        #weather = response.json()
        historic_weather = response_open_meteo.json()
        # df = pd.DataFrame(weather)
        pprint(historic_weather)
        # Connect to your postgres DB
        conn = pg.connect(  dbname="meteo_db",
        user="airflow",  # ou airflow
        password="tsanta",
        host="localhost",
        port="5432")
        
        values = ()
        # Open a cursor to perform database operations
        cur = conn.cursor()


        # Execute a query
        cur.execute("INSERT INTO dim_date (date_id,full_date,day,month,month_name,year,season) VALUES (%s,%s,%s,%s,%s,%s,%s) ON CONFLICT (date_id) DO NOTHING",())        # Retrieve query results
        records = cur.fetchall()
        print(records)
    


def extract_data(cities):
    conn = pg.connect(
        dbname="meteo_db",
        user="airflow",
        password="tsanta",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()

    for city in cities:
        name = city["q"]
        lat = city["latitude"]
        lon = city["longitude"]

        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": "2025-06-01",
            "end_date": "2025-06-03",
            "daily": "sunshine_duration",
            "hourly": [
                "temperature_2m",
                "wind_speed_10m",
                "precipitation",
                "relative_humidity_2m"
            ],
            "timezone": "auto"
        }

        response = requests.get("https://archive-api.open-meteo.com/v1/archive", params=params)
        data = response.json()
        pprint(data)

        # === Sunshine par jour
        daily_dates = data["daily"]["time"]
        daily_sun = data["daily"].get("sunshine_duration", [0] * len(daily_dates))
        sun_by_date = dict(zip(daily_dates, daily_sun))

        # === Extraction des données horaires
        times = data["hourly"]["time"]
        temp = data["hourly"]["temperature_2m"]
        wind = data["hourly"]["wind_speed_10m"]
        humid = data["hourly"]["relative_humidity_2m"]
        precip = data["hourly"]["precipitation"]

        # === Regroupement par date
        daily_data = {}
        for t, t2m, w10m, h2m, p in zip(times, temp, wind, humid, precip):
            date = t.split("T")[0]
            if date not in daily_data:
                daily_data[date] = {
                    "temps": [],
                    "winds": [],
                    "humidity": [],
                    "precip": [],
                }
            daily_data[date]["temps"].append(t2m)
            daily_data[date]["winds"].append(w10m)
            daily_data[date]["humidity"].append(h2m)
            daily_data[date]["precip"].append(p)

        # === dim_city
        cur.execute("SELECT city_id FROM dim_city WHERE city_name = %s", (name,))
        city_row = cur.fetchone()
        if city_row:
            city_id = city_row[0]
        else:
            cur.execute("""
                INSERT INTO dim_city (city_name, country, continent, latitude, longitude)
                VALUES (%s, %s, %s, %s, %s) RETURNING city_id
            """, (name, None, None, lat, lon))
            city_id = cur.fetchone()[0]

        for date_str, values in daily_data.items():
            full_date = datetime.strptime(date_str, "%Y-%m-%d")
            date_id = int(full_date.strftime("%Y%m%d"))
            day = full_date.day
            month = full_date.month
            year = full_date.year
            month_name = calendar.month_name[month]
            season = (
                "Spring" if month in [3, 4, 5]
                else "Summer" if month in [6, 7, 8]
                else "Autumn" if month in [9, 10, 11]
                else "Winter"
            )

            temp_max = max(values["temps"])
            wind_avg = round(np.mean(values["winds"]), 2)
            humid_avg = round(np.mean(values["humidity"]), 2)
            precip_sum = round(sum(values["precip"]), 2)
            sun_hours = round(sun_by_date.get(date_str, 0) / 3600, 2)

            # === weather_score simplifié ===
            score = 0
            if 22 <= temp_max <= 28: score += 3
            elif 18 <= temp_max < 22 or 28 < temp_max <= 32: score += 2
            elif 15 <= temp_max < 18 or 32 < temp_max <= 35: score += 1

            if precip_sum < 1: score += 2
            elif precip_sum < 5: score += 1

            if 40 <= humid_avg <= 70: score += 1
            if wind_avg < 20: score += 1
            if sun_hours > 4: score += 1

            weather_score = score

            # === dim_date
            cur.execute("""
                INSERT INTO dim_date (date_id, full_date, day, month, month_name, year, season)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (date_id) DO NOTHING
            """, (date_id, full_date, day, month, month_name, year, season))

            # === weather_fact
            cur.execute("""
                INSERT INTO weather_fact (
                    date_id, city_id, temp_max, precipitation,
                    wind_speed, humidity, sun_hours, uv_index,
                    weather_score, source
                )
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """, (
                date_id, city_id, temp_max, precip_sum,
                wind_avg, humid_avg, sun_hours, None,
                weather_score, "historical"
            ))

    conn.commit()
    cur.close()
    conn.close()
    print("✅ Données météo insérées correctement.")
extract_data(cities)

# ###print(dict(tana_weather))
# pprint(tana_weather)
# df = pd.DataFrame(tana_weather)
# print(df)


{'daily': {'sunshine_duration': [34327.04, 37136.54, 38201.31],
           'time': ['2025-06-01', '2025-06-02', '2025-06-03']},
 'daily_units': {'sunshine_duration': 's', 'time': 'iso8601'},
 'elevation': 2557.0,
 'generationtime_ms': 25.749564170837402,
 'hourly': {'precipitation': [0.0,
                              0.0,
                              0.0,
                              0.0,
                              0.0,
                              0.0,
                              0.0,
                              0.0,
                              0.0,
                              0.4,
                              0.3,
                              0.6,
                              0.5,
                              0.2,
                              0.2,
                              0.2,
                              0.2,
                              0.2,
                              0.0,
                              0.0,
                              0.0,
          